In [1]:
def procrustes(X, Y, scaling=True, reflection='best'):
    """
    A port of MATLAB's `procrustes` function to Numpy.

    Procrustes analysis determines a linear transformation (translation,
    reflection, orthogonal rotation and scaling) of the points in Y to best
    conform them to the points in matrix X, using the sum of squared errors
    as the goodness of fit criterion.

        d, Z, [tform] = procrustes(X, Y)

    Inputs:
    ------------
    X, Y    
        matrices of target and input coordinates. they must have equal
        numbers of  points (rows), but Y may have fewer dimensions
        (columns) than X.

    scaling 
        if False, the scaling component of the transformation is forced
        to 1

    reflection
        if 'best' (default), the transformation solution may or may not
        include a reflection component, depending on which fits the data
        best. setting reflection to True or False forces a solution with
        reflection or no reflection respectively.

    Outputs
    ------------
    d       
        the residual sum of squared errors, normalized according to a
        measure of the scale of X, ((X - X.mean(0))**2).sum()

    Z
        the matrix of transformed Y-values

    tform   
        a dict specifying the rotation, translation and scaling that
        maps X --> Y

    """

    n,m = X.shape
    ny,my = Y.shape

    muX = X.mean(0)
    muY = Y.mean(0)

    X0 = X - muX
    Y0 = Y - muY

    ssX = (X0**2.).sum()
    ssY = (Y0**2.).sum()

    # centred Frobenius norm
    normX = np.sqrt(ssX)
    normY = np.sqrt(ssY)

    # scale to equal (unit) norm
    X0 /= normX
    Y0 /= normY

    if my < m:
        Y0 = np.concatenate((Y0, np.zeros(n, m-my)),0)

    # optimum rotation matrix of Y
    A = np.dot(X0.T, Y0)
    U,s,Vt = np.linalg.svd(A,full_matrices=False)
    V = Vt.T
    T = np.dot(V, U.T)

    if reflection is not 'best':

        # does the current solution use a reflection?
        have_reflection = np.linalg.det(T) < 0

        # if that's not what was specified, force another reflection
        if reflection != have_reflection:
            V[:,-1] *= -1
            s[-1] *= -1
            T = np.dot(V, U.T)

    traceTA = s.sum()

    if scaling:

        # optimum scaling of Y
        b = traceTA * normX / normY

        # standarised distance between X and b*Y*T + c
        d = 1 - traceTA**2

        # transformed coords
        Z = normX*traceTA*np.dot(Y0, T) + muX

    else:
        b = 1
        d = 1 + ssY/ssX - 2 * traceTA * normY / normX
        Z = normY*np.dot(Y0, T) + muX

    # transformation matrix
    if my < m:
        T = T[:my,:]
    c = muX - b*np.dot(muY, T)

    #transformation values 
    tform = {'rotation':T, 'scale':b, 'translation':c}

    return d, Z, tform

In [2]:
import pandas as pd
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 

from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

from sklearn.model_selection import train_test_split
import json

torch.set_printoptions(linewidth=120) #Display options for output
torch.set_grad_enabled(True) #Already on by default


In [3]:
def get_device():
    if torch.cuda.is_available():
        device = 'cuda:0'
    else:
        device = 'cpu'
    return device
device = get_device()

In [4]:
device

'cuda:0'

In [5]:
print(torch.__version__)
print(torchvision.__version__)

1.5.1+cu101
0.6.1+cu101


# Load and Prepare Data Train set and validationset

In [6]:
#calculate number of correct prediction given labels
def get_num_correct(preds,labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [7]:
def preprocess_data():
    normalized_data = pd.read_csv('dystonia_dataset_train_normalized.csv')
    
    X = normalized_data['image']
    Y = normalized_data['class']

    X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=3)
    
    X_train_Compiled = np.empty((0,50,51,1))
    X_test_Compiled = np.empty((0,50,51,1))
    Y_test_compiled = []
    Y_train_compiled = []
    
    for index in X_train.index:
        json_load = json.loads(X_train[index])
        a_restored = np.asarray(json_load)
        X_train_Compiled=np.append(X_train_Compiled,[a_restored],axis=0)
        X_train_Compiled=X_train_Compiled.astype(np.float32)
        
    for index in X_test.index:
        json_load = json.loads(X_test[index])
        a_restored = np.asarray(json_load)
        X_test_Compiled=np.append(X_test_Compiled,[a_restored],axis=0)
        X_test_Compiled=X_test_Compiled.astype(np.float32)
        
    for index in Y_test.index:
        Y_test_compiled.append(Y_test[index])
        
    for index in Y_train.index:
        Y_train_compiled.append(Y_train[index])
        
    return X_train_Compiled,Y_train_compiled,X_test_Compiled,Y_test_compiled

In [8]:
train_images,train_labels,test_images,test_labels = preprocess_data()

In [9]:
train_mean = train_images.mean()
train_std = train_images.std()

test_mean = test_images.mean()
test_std = test_images.std()

In [10]:
train_mean,train_std

(-0.009724988, 0.27727515)

In [11]:
test_mean,test_std

(-0.008603053, 0.27681193)

# Preprocess Test Data (referred as validationset)

In [12]:
def preprocess_test_data():
    normalized_data = pd.read_csv('dystonia_dataset_test_normalized.csv')

    X_test = normalized_data['image']
    Y_test = normalized_data['class']

    X_test_Compiled = np.empty((0,50,51,1))
    Y_test_Compiled = []

        
    for index in X_test.index:
        json_load = json.loads(X_test[index])
        a_restored = np.asarray(json_load)
        X_test_Compiled=np.append(X_test_Compiled,[a_restored],axis=0)
        X_test_Compiled=X_test_Compiled.astype(np.float32)
        
    for index in Y_test.index:
        Y_test_Compiled.append(Y_test[index])
            
    return X_test_Compiled,Y_test_Compiled

In [13]:
val_images, val_labels = preprocess_test_data()

In [14]:
val_mean = val_images.mean()
val_std = val_images.std()

In [15]:
val_mean,val_std

(-0.006763445, 0.27508262)

In [16]:
# custom dataset
class DystoniaDataset(Dataset):
    
    def __init__(self, images, labels=None, transforms=None):
        self.X = images
        self.y = labels
        self.transforms = transforms
        self.targets = []
        self.classes = ['0 GDS','1 GDS','2 GDS']

    
    @property
    def train_labels(self):
        self.targets = train_labels
        return self.targets

    @property
    def test_labels(self): 
        self.targets = test_labels
        return self.targets
    
    @property
    def val_labels(self): 
        self.targets = val_labels
        return self.targets
    
    def __len__(self):
        return (len(self.X))
    
    def __getitem__(self, i):
        data = self.X[i]
        if self.transforms:
            data = self.transforms(data)
            
        if self.y is not None:
            return (data, self.y[i])
        else:
            return data

In [17]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5)
        
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5)
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5)
        
        self.bn1 = nn.BatchNorm2d(num_features=16)
        self.bn2 = nn.BatchNorm2d(num_features=32)
        self.bn3 = nn.BatchNorm2d(num_features=64)
        self.bn4 = nn.BatchNorm2d(num_features=128)
        self.bn5 = nn.BatchNorm1d(num_features=1000)
            
        self.dropout1 = nn.Dropout(0.5)
        self.dropout2 = nn.Dropout(0.5)
        self.dropout3 = nn.Dropout(0.5)

        self.fc1 = nn.Linear(in_features=128 * 6 * 6, out_features=1000)
        self.fc2 = nn.Linear(in_features=1000, out_features=100)
        self.out = nn.Linear(in_features=100, out_features=3)

    def forward(self, t):
        # (1) input layer
        t = t
#         print('INput layer', t)
#         print('Input layer Shpae', t.shape)

        # (2) hidden conv layer
        t = self.conv1(t)
        t = F.relu(t)
        t = self.bn1(t)
        
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        t = self.bn2(t)
        
        # dropout 0.3
        t = self.dropout1(t)

        # (3) hidden conv layer
        t = self.conv3(t)
        t = F.relu(t)
        t = self.bn3(t)
        
        t = self.conv4(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        t = self.bn4(t)


        # dropout 0.4
        t = self.dropout2(t)

        # (4) hidden linear layer
        t = t.reshape(-1, 128 * 6 * 6)
        t = self.fc1(t)
        t = F.relu(t)
        t = self.bn5(t)

        # dropout 0.5
        t = self.dropout3(t)

        # (5) hidden linear layer
        t = self.fc2(t)
        t = F.relu(t)

        # (6) output layer
        t = self.out(t)
        #t = F.softmax(t, dim=1)

        return t

In [18]:
# define transforms
train_transform = transforms.Compose([
     transforms.ToTensor(),
     transforms.Normalize([train_mean],[train_std])
])

test_transform = transforms.Compose([
     transforms.ToTensor(),
     transforms.Normalize([test_mean],[test_std])
])

val_transform = transforms.Compose([
     transforms.ToTensor(),
     transforms.Normalize([val_mean],[val_std])
])

In [19]:
train_set = DystoniaDataset(train_images, train_labels, train_transform)
test_set = DystoniaDataset(test_images, test_labels, test_transform)
val_set = DystoniaDataset(val_images, val_labels, val_transform)

In [20]:
#Dataloaders
train_loader = DataLoader(train_set, batch_size=64, shuffle=True, num_workers=1)
test_loader = DataLoader(test_set, batch_size=64, shuffle=True, num_workers=1)
val_loader = DataLoader(val_set, batch_size=64, shuffle=True, num_workers=1)

# Network Architecture

In [21]:
network = Network().to(device)
print(network)

Network(
  (conv1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv4): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn5): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout1): Dropout(p=0.5, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
  (dropout3): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=4608, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=100, bias=True)
  (out): Linear(in_features=100, out_fea

In [22]:
# loss
# criterion = nn.CrossEntropyLoss()
# optimizer
# optimizer = optim.SGD(network.parameters(), lr=0.001, momentum=0.9)
# optimizer = optim.Adam(network.parameters(),lr=0.01)

In [23]:
def get_all_preds(model,loader):
    all_preds = torch.tensor([]).to(device)
    for batch in loader:
        images = batch[0].to(device,non_blocking=True)
        labels = batch[1].to(device,non_blocking=True)
        
        preds = model(images)
        all_preds = torch.cat(
        (all_preds,preds),
            dim=0
        )
    return all_preds

In [37]:
def train(network,train_loader):
    
    optimizer = optim.Adam(network.parameters(),lr=0.0001)
    
    for epoch in range(200):
        total_loss = 0
        total_correct = 0

        for batch in train_loader:
            images = batch[0].to(device,non_blocking=True)
            labels = batch[1].to(device,non_blocking=True)
            
            #Forward Pass
            preds = network(images) #pass Batch to the network and get preds tensor
            loss = F.cross_entropy(preds,labels) #calculate loss tensor
#             loss = criterion(preds, labels)

            #Backward Pass and optimize
            #https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch
            optimizer.zero_grad() #zero our gradients 
        
            #Calculating Gradients 
            loss.backward() 
            
            #Update Weights (step in the direction of loss funtions minimum:)
            #optimizer updates the networks weight using gradients and learning rate
            optimizer.step() 
            
            total_loss += loss.item()
            total_correct += get_num_correct(preds,labels)
        
        print('epoch: ', epoch, 'total_correct', total_correct, 'loss', total_loss)
    print('Done Training')
    
def test(network, test_loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data[0].to(device, non_blocking=True), data[1].to(device, non_blocking=True)
            outputs = network(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('Accuracy of the network on test images: %0.3f %%' % (
        100 * correct / total))

In [38]:
def test_new(network,test_loader):
    with torch.no_grad():
        prediction_loader = torch.utils.data.DataLoader(test_set,batch_size=64)
        test_preds = get_all_preds(network,prediction_loader)
    targets = torch.tensor(test_set.test_labels).to(device)
    preds_correct = get_num_correct(test_preds, targets)
    print('Total Correct: ', preds_correct)
    print('test accuracy', preds_correct / len(test_set))
    
def val_new(network,val_loader):
    with torch.no_grad():
        prediction_loader = torch.utils.data.DataLoader(val_set,batch_size=64)
        val_preds = get_all_preds(network,prediction_loader)
    targets = torch.tensor(val_set.val_labels).to(device)
    preds_correct = get_num_correct(val_preds, targets)
    print('Total Correct: ', preds_correct)
    print('val accuracy', preds_correct / len(val_set)) 

In [39]:
train(network, train_loader)

epoch:  0 total_correct 479 loss 0.22821826115250587
epoch:  1 total_correct 482 loss 0.08344553271308541
epoch:  2 total_correct 482 loss 0.07037812017370015
epoch:  3 total_correct 483 loss 0.05028401443269104
epoch:  4 total_correct 484 loss 0.013520664244424552
epoch:  5 total_correct 483 loss 0.05038601323030889
epoch:  6 total_correct 482 loss 0.10596986627206206
epoch:  7 total_correct 483 loss 0.06420086231082678
epoch:  8 total_correct 483 loss 0.032667651772499084
epoch:  9 total_correct 484 loss 0.01228137977886945
epoch:  10 total_correct 483 loss 0.021238770335912704
epoch:  11 total_correct 482 loss 0.06439856020733714
epoch:  12 total_correct 482 loss 0.07950056972913444
epoch:  13 total_correct 484 loss 0.013687115162611008
epoch:  14 total_correct 484 loss 0.01967987697571516
epoch:  15 total_correct 484 loss 0.0208441954528098
epoch:  16 total_correct 482 loss 0.06327204452827573
epoch:  17 total_correct 483 loss 0.026618073927238584
epoch:  18 total_correct 484 loss 

epoch:  150 total_correct 483 loss 0.06090780906379223
epoch:  151 total_correct 484 loss 0.005054345980170183
epoch:  152 total_correct 484 loss 0.02005778905004263
epoch:  153 total_correct 484 loss 0.006210735999047756
epoch:  154 total_correct 484 loss 0.016921856336921337
epoch:  155 total_correct 484 loss 0.016542718978598714
epoch:  156 total_correct 482 loss 0.0680966661311686
epoch:  157 total_correct 484 loss 0.005243563326075673
epoch:  158 total_correct 484 loss 0.011887714674230665
epoch:  159 total_correct 484 loss 0.004370710710645653
epoch:  160 total_correct 483 loss 0.03293235274031758
epoch:  161 total_correct 484 loss 0.0039056282839737833
epoch:  162 total_correct 484 loss 0.016846611397340894
epoch:  163 total_correct 484 loss 0.014256746391765773
epoch:  164 total_correct 484 loss 0.007072289357893169
epoch:  165 total_correct 484 loss 0.005943969823420048
epoch:  166 total_correct 483 loss 0.06987468525767326
epoch:  167 total_correct 483 loss 0.0387153141200542

In [29]:
# test(network, test_loader)

In [30]:
# test_new(network, test_loader)

In [31]:
# val_new(network,val_loader)

In [131]:
with torch.no_grad():
    prediction_loader = torch.utils.data.DataLoader(train_set,batch_size=64)
    train_preds = get_all_preds(network,prediction_loader)
targets = torch.tensor(train_set.train_labels).to(device)
preds_correct = get_num_correct(train_preds, targets)
print('Total Correct: ', preds_correct)
print('Total Train Set', len(train_set))
print('accuracy', preds_correct / len(train_set)) 

Total Correct:  484
Total Train Set 484
accuracy 1.0


In [132]:
with torch.no_grad():
    prediction_loader = torch.utils.data.DataLoader(test_set,batch_size=64)
    test_preds = get_all_preds(network,prediction_loader)
targets = torch.tensor(test_set.test_labels).to(device)
preds_correct = get_num_correct(test_preds, targets)
print('Total Correct: ', preds_correct)
print('Total test set', len(test_set))
print('accuracy', preds_correct / len(test_set)) 

Total Correct:  104
Total test set 121
accuracy 0.859504132231405


In [133]:
with torch.no_grad():
    prediction_loader = torch.utils.data.DataLoader(val_set,batch_size=64)
    val_preds = get_all_preds(network,prediction_loader)
targets = torch.tensor(val_set.val_labels).to(device)
preds_correct = get_num_correct(val_preds, targets)
print('Total Correct: ', preds_correct)
print('Total val set', len(val_set))
print('val accuracy', preds_correct / len(val_set)) 

Total Correct:  24
Total val set 60
val accuracy 0.4


# Building a Confusion Matrix for Train set

In [134]:
# train_set.targets
targets = torch.tensor(train_set.train_labels).to(device)

stacked = torch.stack(
    (
        targets,
        train_preds.argmax(dim=1)
    ),
    dim=1
)
cmt = torch.zeros(3,3,dtype=torch.int32)

print('Initialized Confusion Matrix')
print(cmt)
print('\n')
for p in stacked:
    tl, pl = p.tolist()
    cmt[tl,pl] = cmt[tl,pl] + 1
print('Confusion Matrix for Train Set')
cmt

Initialized Confusion Matrix
tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]], dtype=torch.int32)


Confusion Matrix for Train Set


tensor([[157,   0,   0],
        [  0, 157,   0],
        [  0,   0, 170]], dtype=torch.int32)

# Building Confusion Matrix for validation set

In [135]:
targets = torch.tensor(test_set.test_labels).to(device)

stacked = torch.stack(
    (
        targets,
        test_preds.argmax(dim=1)
    ),
    dim=1
)
cmt = torch.zeros(3,3,dtype=torch.int32)
for p in stacked:
    j,k = p.tolist()
    cmt[j,k] = cmt[j,k] + 1
print('Confusion Matrix for Validation set')
cmt

Confusion Matrix for Validation set


tensor([[45,  1,  0],
        [ 3, 33,  7],
        [ 2,  4, 26]], dtype=torch.int32)

# Building COnfusion Matrix for Test Set (here named as val_set)

In [136]:
targets = torch.tensor(val_set.val_labels).to(device)

stacked = torch.stack(
    (
        targets,
        val_preds.argmax(dim=1)
    ),
    dim=1
)
cmt = torch.zeros(3,3,dtype=torch.int32)
for p in stacked:
    j,k = p.tolist()
    cmt[j,k] = cmt[j,k] + 1
print('Confusion Matrix for Validation set')
cmt

Confusion Matrix for Validation set


tensor([[ 8, 10,  2],
        [ 5,  2, 13],
        [ 0,  6, 14]], dtype=torch.int32)

# Precision, Recall and F1 Score of indiviuals of validation set (here test set)

There are three labels 0,1,2 for gps of 0, 1 , 5 respectively. Change value gds level 0 to any other value for calculating individuals
Modified from: # https://stackoverflow.com/questions/56643503/efficient-metrics-evaluation-in-pytorch

In [137]:
target_true = 0
predicted_true=0
correct_true=0
gds_label = 2 #0, 1, 2 class

targets = torch.tensor(test_set.test_labels).to(device)
predicted_classes = torch.argmax(test_preds, dim=1) == gds_label
target_classes = targets == gds_label


target_true += torch.sum(targets == gds_label).float()
predicted_true += torch.sum(predicted_classes).float()
correct_true += torch.sum(target_classes * predicted_classes == True)
# print(correct_true)

In [138]:
recall = correct_true / target_true
precision = correct_true / predicted_true
f1_score = 2 * (precision * recall)/ (precision + recall)

In [139]:
recall

tensor(0.8125, device='cuda:0')

In [129]:
precision

tensor(0.8182, device='cuda:0')

In [130]:
f1_score

tensor(0.8308, device='cuda:0')

In [102]:
# predicted_classes

In [103]:
predicted_true

tensor(47., device='cuda:0')

In [104]:
correct_true

tensor(43, device='cuda:0')

In [105]:
target_true

tensor(46., device='cuda:0')

# Precision, Recall and F1 Score of Test set here referred as val set

In [116]:
target_true = 0
predicted_true=0
correct_true=0
gds_label = 2 #0, 1, 2 class

targets = torch.tensor(val_set.val_labels).to(device)
predicted_classes = torch.argmax(val_preds, dim=1) == gds_label
target_classes = targets == gds_label


target_true += torch.sum(targets == gds_label).float()
predicted_true += torch.sum(predicted_classes).float()
correct_true += torch.sum(target_classes * predicted_classes == True)
# print(correct_true)

In [117]:
recall = correct_true / target_true
precision = correct_true / predicted_true
f1_score = 2 * (precision * recall)/ (precision + recall)

In [118]:
recall

tensor(0.4500, device='cuda:0')

In [119]:
precision

tensor(0.4286, device='cuda:0')

In [120]:
f1_score

tensor(0.4390, device='cuda:0')

In [74]:
target_true

tensor(20., device='cuda:0')

In [ ]:
predicted_true

In [ ]:
correct_true